In [7]:
using BenchmarkTools
include("../src/MyUnits.jl")

x_arr = 10 .^ (range(-2, stop=2, length=50))

function f(x::Array, y::Real)
    x .+ y
end

function f(x::Array, y::Array)
    x + y
end

f (generic function with 2 methods)

In [8]:
@benchmark f(x_arr, x_arr)

BenchmarkTools.Trial: 10000 samples with 990 evaluations.
 Range (min … max):  41.457 ns … 825.884 ns  ┊ GC (min … max):  0.00% … 92.20%
 Time  (median):     48.022 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   55.755 ns ±  56.828 ns  ┊ GC (mean ± σ):  10.40% ±  9.36%

  █▆▃▂                                                         ▁
  ████▇▇▆▆▅▅▄▄▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ █
  41.5 ns       Histogram: log(frequency) by time       552 ns <

 Memory estimate: 496 bytes, allocs estimate: 1.

In [1]:
using BenchmarkTools

using Unitful
import Unitful: km, s, Gyr
using UnitfulAstro: Mpc, Gpc, Msun

include("../src/MyCosmology.jl")
include("../src/PowerSpectrum.jl")

import .MyCosmology: BASEPLANCK18


MyCosmology.z_eq_Λm()
#@benchmark cosmo = MyCosmology.BaseFlatLCDM(0.7, 0.7, 0.3)


#MyCosmology.temperature_cmb(cosmo)
#ps = PowerSpectrum.PowerSpectrumLCDM(1e-9, 0.96)
#ps.value(10.0 / Mpc )

#H0::Quantity{Real} = 70.0u"km/s/Mpc"

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.972 μs …   5.921 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.037 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.083 μs ± 157.382 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▆▂ ▅▇▄▁▆▆▃▂ ▂▁▂▁   ▁    ▅▅▁         ▂                     ▂
  ████▆█████████████▇▇██████████▆▆██▇█▇▇██▆▃▄▅▄▄▄▅▄▄█▆▅▄▅▃▅▃▄ █
  2.97 μs      Histogram: log(frequency) by time      3.55 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [2]:
using Plots

z = 10 .^(range(-2,stop=5,length=100))

println(BASEPLANCK18.Ω_m0, " ", BASEPLANCK18.Ω_r0, " ", BASEPLANCK18.Ω_Λ0)

Ω_r = ρ_r(BASEPLANCK18, z) ./ MyCosmology.ρ_c(BASEPLANCK18, z)
Ω_m = MyCosmology.Ω_m(BASEPLANCK18, z)
Ω_Λ = MyCosmology.Ω_Λ(BASEPLANCK18, z)

ρ_r = MyCosmology.ρ_r(BASEPLANCK18, z)
ρ_m = MyCosmology.ρ_m(BASEPLANCK18, z)
ρ_Λ = MyCosmology.ρ_Λ(BASEPLANCK18, z)
ρ_c = MyCosmology.ρ_c(BASEPLANCK18, z)

plot(z, Ω_r)
plot!(z, Ω_m)
plot!(z, Ω_Λ)


plot!(size=(600, 400), xscale=:log10,  minorgrid=false)


0.31377 9.21221186221331e-5 0.6861378778813779


ErrorException: type BaseFlatLCDM has no field ρ_r